In [1]:
import time
import torch
from transformers import GPTJForCausalLM, GPT2Tokenizer
import torch

/home/azureuser/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Will need at least 13-14GB of Vram for CUDA
if torch.cuda.is_available():
    model =  GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B",cache_dir="/datadrive/cache_dir/", torch_dtype=torch.float16).cuda()
else:
    print("GPU not available, check driver and cuda installation and reboot if neccessary")
    #model =  GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", torch_dtype=torch.float16)

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [4]:
environment_description = """
Name: Max  Occupation: Consultant Salary: 500
Name: Sam  Occupation: Consultant Salary: 600
Name: James Occupation: Consultant Salary: 700"""

In [5]:
input_text = environment_description
input_ids = tokenizer.encode(str(input_text), return_tensors='pt').cuda()

output = model.generate(
    input_ids,
    do_sample=True,
    top_p=0.99,
    top_k=10,
    temperature=0.9,
    max_new_tokens=200,
    #penalty_alpha=0.6

)

print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Name: Max  Occupation: Consultant Salary: 500
Name: Sam  Occupation: Consultant Salary: 600
Name: James Occupation: Consultant Salary: 700

I want to create a query which takes Max's salary and the total number of Consultants in the list and then return the names of all the consultants who earn more than Max.
I have tried with following query, but it returns only one result.
WITH cte AS(  
  SELECT Name, Occupation, Salary, ROW_NUMBER() OVER (PARTITION BY Name) rn 
  FROM Employee
)
SELECT Name, Occupation, Salary
FROM cte
WHERE rn = 1

A:

You can use subquery:
SELECT *
FROM ( 
  SELECT *
  FROM Employee
) t
WHERE Salary > ( SELECT MAX( Salary ) FROM Employee )




In [6]:
from flask import Flask, request
from flask_restful import Resource, Api


app = Flask(__name__)
api = Api(app)

class generate_text(Resource):
    def get(self, text, tokens):

        environment_description = text

        input_text = environment_description
        input_ids = tokenizer.encode(str(input_text), return_tensors='pt').cuda()

        output = model.generate(
            input_ids,
            do_sample=True,
            top_p=0.99,
            top_k=10,
            temperature=0.9,
            max_new_tokens=int(tokens)*10,
            penalty_alpha=0.6
            )

        out = tokenizer.decode(output[0], skip_special_tokens=True)
        
        
        result = {"result":out}
        return result   
        
api.add_resource(generate_text, '/generate/<text>/<tokens>') # Route_1

app.run(port='5002')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002 (Press CTRL+C to quit)
127.0.0.1 - - [11/Nov/2022 00:51:19] "GET /generate/Question:%20Fetch%20the%20companies%20that%20have%20less%20than%20five%20people%20in%20it.%0A%20%20%20%20%20%20%20%20%20%20%20%20Answer:%20SELECT%20COMPANY,%20COUNT(EMPLOYEE_ID)%20FROM%20Employee%20GROUP%20BY%20COMPANY%20HAVING%20COUNT(EMPLOYEE_ID)%20%3C%205;%0A%20%20%20%20%20%20%20%20%20%20%20%20 HTTP/1.1" 404 -


In [7]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/generate', methods=['GET']) 
def generate():
    data = request.json
    
    return jsonify(data)

app.run(port='5002')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002 (Press CTRL+C to quit)
127.0.0.1 - - [11/Nov/2022 00:59:45] "GET /generate HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2022 01:00:05] "GET /generate HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2022 01:00:31] "GET /generate HTTP/1.1" 200 -


In [11]:
from flask import Flask, request, jsonify
import json

app = Flask(__name__)

@app.route('/generate', methods=['GET']) 
def generate():


    data = json.loads(request.json, strict=False)
    print(data)

    environment_description = data["body"]

    input_text = environment_description
    input_ids = tokenizer.encode(str(input_text), return_tensors='pt').cuda()

    output = model.generate(
        input_ids,
        do_sample=True,
        top_p=0.99,
        top_k=10,
        temperature=0.9,
        max_new_tokens=int(data['token'])*10,
        penalty_alpha=0.6
        )

    out = tokenizer.decode(output[0], skip_special_tokens=True)
    
    
    result = {"result":out}
    
    return jsonify(result)

app.run(port='5002')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002 (Press CTRL+C to quit)
[2022-11-11 01:16:13,906] ERROR in app: Exception on /generate [GET]
Traceback (most recent call last):
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/tmp/ipykernel_48685/1037202936.py", line 8, in generate
    data = json.loads(request.json)
  File "/home/azureuser/anaconda3/envs/ldm/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_de

{'body': 'This is a testhhh Answer:', 'token': '10'}


127.0.0.1 - - [11/Nov/2022 01:18:51] "GET /generate HTTP/1.1" 200 -
[2022-11-11 01:19:35,284] ERROR in app: Exception on /generate [GET]
Traceback (most recent call last):
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/azureuser/.local/lib/python3.8/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/tmp/ipykernel_48685/1037202936.py", line 8, in generate
    data = json.loads(request.json)
  File "/home/azureuser/anaconda3/envs/ldm/lib/python3.8/json/__init__.py", line 357, in loads
    return _d